In [4]:
import pandas as pd
import numpy as np

# df_dropout=pd.read_csv('./acgr19.csv')
# acgr represented adjusted cohort graduation rate
# https://www3.cde.ca.gov/demo-downloads/acgr/acgr21.txt
df_dropout=pd.read_csv('./acgr21.csv')


In [12]:
df_dropout.head()

,AcademicYear,AggregateLevel,CountyCode,Dcode,SchoolCode,CountyName,DistrictName,SchoolName,CharterSchool,DASS,...,SPED Certificate (Count),SPED Certificate (Rate),GED Completer (Count),GED Completer (Rate),Other Transfer (Count),Other Transfer (Rate),Dropout (Count),Dropout (Rate),Still Enrolled (Count),Still Enrolled (Rate)
8531,2020-21,D,1,10017,0.0,Alameda,Alameda County Office of Education,District Office,All,All,...,0,0.0,0,0.0,0,0.0,35,21.6,21,13.0
8532,2020-21,D,1,10017,0.0,Alameda,Alameda County Office of Education,District Office,All,All,...,0,0.0,0,0.0,0,0.0,75,41.0,22,12.0
8533,2020-21,D,1,10017,0.0,Alameda,Alameda County Office of Education,District Office,All,All,...,*,*,*,*,*,*,*,*,*,*
8534,2020-21,D,1,10017,0.0,Alameda,Alameda County Office of Education,District Office,All,All,...,0,0.0,0,0.0,0,0.0,45,36.6,15,12.2
8535,2020-21,D,1,10017,0.0,Alameda,Alameda County Office of Education,District Office,All,All,...,*,*,*,*,*,*,*,*,*,*


In [11]:
# drop data without district code and change district code to str type for easy merge
df_dropout = df_dropout.dropna(subset=['DistrictCode'])
df_dropout['DistrictCode'] = df_dropout['DistrictCode'].astype(int).astype(str)

# rename to keep consistency
df_dropout.rename(columns={'DistrictCode': 'Dcode'}, inplace=True)


In [13]:
# https://www3.cde.ca.gov/fiscal-downloads/sacs_data/2020-21/sacs2021.exe
# This is a database file consists of several tables. To save, use db software such as Access to export
# For larger files, may need to export to txt. See this readme file: https://www.cde.ca.gov/ds/fd/fd/sacsreadme2122.asp

# Read UserGL.txt with specified column names
# Fixed school code issue
df_UserGL = pd.read_csv('./UserGL.txt', names=['Ccode', 'Dcode', 'SchoolCode', 'Fiscalyear', 'Period', 'Colcode', 'Account', 'Fund', 'Resource', 'Projectyear', 'Goal', 'Function', 'Object', 'Value'], dtype={'SchoolCode': str, 'Dcode': str})


C:\Users\yiqiu\AppData\Local\Temp\ipykernel_47896\3740323859.py:7: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df_UserGL = pd.read_csv('./UserGL.txt', names=['Ccode', 'Dcode', 'SchoolCode', 'Fiscalyear', 'Period', 'Colcode', 'Account', 'Fund', 'Resource', 'Projectyear', 'Goal', 'Function', 'Object', 'Value'], dtype={'SchoolCode': str, 'Dcode': str})


In [14]:
df_UserGL.head()
# each row is for funding assignment
# aggregation based on school name?

,Ccode,Dcode,SchoolCode,Fiscalyear,Period,Colcode,Account,Fund,Resource,Projectyear,Goal,Function,Object,Value
0,1,10017,0000000,2020,A,BA,0100000000000008011,1,0,0,0000,0,8011,9274727.22
1,1,10017,0000000,2020,A,BA,0100000000000008021,1,0,0,0000,0,8021,119323.03
2,1,10017,0000000,2020,A,BA,0100000000000008022,1,0,0,0000,0,8022,1.47
3,1,10017,0000000,2020,A,BA,0100000000000008029,1,0,0,0000,0,8029,2124.98
4,1,10017,0000000,2020,A,BA,0100000000000008041,1,0,0,0000,0,8041,18967582.92


In [15]:
df_UserGL.dtypes

Ccode            int64
Dcode           object
SchoolCode      object
Fiscalyear       int64
Period          object
Colcode         object
Account         object
Fund             int64
Resource         int64
Projectyear     object
Goal            object
Function         int64
Object          object
Value          float64
dtype: object

In [6]:
# Group df_UserGL by Dcode, create a dictionary
fund_by_district=df_UserGL.groupby('Dcode')['Value'].sum()

In [16]:
# Because some data are suppressed for privacy issue, we drop those data
df_dropout['Dropout (Rate)'] = pd.to_numeric(df_dropout['Dropout (Rate)'], errors='coerce')
df_dropout_clean = df_dropout.dropna(subset=['Dropout (Rate)'])
average_dropout_rate_by_district = df_dropout_clean.groupby('Dcode')['Dropout (Rate)'].mean()

In [11]:
avg_dropout_rate_by_district = pd.DataFrame({'Dcode': average_dropout_rate_by_district.index, 'avg_dropout_rate': average_dropout_rate_by_district.values})

total_fund_by_district=pd.DataFrame({'Dcode': fund_by_district.index, 'total_fund':fund_by_district.values})


In [13]:
total_fund_by_district['Dcode']=total_fund_by_district['Dcode'].astype(int).astype(str)

In [14]:
merged_df = pd.merge(avg_dropout_rate_by_district, total_fund_by_district, on='Dcode')

In [15]:
merged_df.head(20)

,Dcode,avg_dropout_rate,total_fund
0,10017,37.480583,3.167240e+08
1,10033,0.000000,5.113658e+07
2,10041,26.326994,3.908802e+08
3,10058,12.817857,5.876958e+07
4,10066,36.400000,5.841169e+07
5,10074,22.698701,3.381559e+08
6,10082,19.997222,3.103624e+07
7,10090,26.217073,6.574781e+08
8,10108,45.312302,9.427600e+08
9,10116,65.806452,9.035249e+07


In [20]:
# https://www.cde.ca.gov/ds/fd/ec/documents/currentexpense2021.xlsx
# Save the first sheet as csv, manually change CDS to Dcode
df_expense=pd.read_csv('./currentexpense2021_by_district.csv')
df_expense.head()

,CO,Dcode,DISTRICT,EDP 365,Current\nExpense ADA,Current\nExpense Per ADA,LEA Type
0,1,61119,Alameda Unified,"122,421,584.91","8,975.20","13,639.98",Unified
1,1,61127,Albany City Unified,"47,343,135.51","3,484.19","13,587.99",Unified
2,1,61143,Berkeley Unified,"173,753,016.19","9,426.03","18,433.32",Unified
3,1,61150,Castro Valley Unified,"105,286,407.91","8,976.77","11,728.76",Unified
4,1,61168,Emery Unified,"12,731,320.51",688.60,"18,488.70",Unified


In [21]:
# Check for duplicate Dcode in df_expense
if df_expense['Dcode'].duplicated().any():
    print("There are duplicate Dcode in df_expense")
else:
    print("There are no duplicate Dcode in df_expense")

df_expense['Dcode']=df_expense['Dcode'].astype(int).astype(str)


There are no duplicate Dcode in df_expense


In [18]:
# merge things on Dcode
dropout_fund_cost = pd.merge(merged_df, df_expense, on='Dcode')

dropout_fund_cost.head()


,Dcode,avg_dropout_rate,total_fund,CO Code,District,EDP 365,Current Expense ADA,Current Expense Per ADA,LEA Type
0,61119,8.892230,7.523954e+08,1,Alameda Unified,"117,225,882.50","8,968.85","13,070.34",Unified
1,61127,2.297727,4.821404e+08,1,Albany City Unified,"46,611,059.59","3,544.52","13,150.18",Unified
2,61143,12.977778,1.264753e+09,1,Berkeley Unified,"159,457,818.49","9,356.44","17,042.57",Unified
3,61150,5.779268,8.022681e+08,1,Castro Valley Unified,"102,239,937.34","8,940.20","11,435.98",Unified
4,61168,0.000000,8.132166e+07,1,Emery Unified,"12,504,023.21",681.82,"18,339.19",Unified


In [35]:

# df=pd.read_csv('./dropout_fund_cost.csv',thousands=',')
# columns = ['avg_dropout_rate', 'total_fund', 'EDP 365', 'Current Expense ADA', 'Current Expense Per ADA','TOTALREV','TFEDREV']
# import pandas as pd
# import seaborn as sns
# import matplotlib.pyplot as plt

# # Calculate the correlation matrix
# corr_matrix = df[columns].corr()

# # Create a heatmap of the correlation matrix
# sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')

# # Show the plot
# plt.show()


Unnamed: 0                   int64
Dcode                        int64
avg_dropout_rate           float64
total_fund                 float64
CO Code                      int64
District                    object
EDP 365                    float64
Current Expense ADA        float64
Current Expense Per ADA    float64
LEA Type                    object
TOTALREV                     int64
TFEDREV                      int64
C14                          int64
C15                          int64
C16                          int64
C17                          int64
C19                          int64
B11                          int64
C20                          int64
C25                          int64
C36                          int64
B10                          int64
B12                          int64
B13                          int64
TSTREV                       int64
C01                          int64
C04                          int64
C05                          int64
C06                 

ADA means average daily attendence. It is unexpected that the correlation of total funding assigned to each school district has no correlation with the dropout rate. We might need to look at funding per school and dropout rate variance to refine our analysis. 

In [44]:
df[columns].describe()
# Format issue of dropout rate

,avg_dropout_rate,total_fund,EDP 365,Current Expense ADA,Current Expense Per ADA,TOTALREV,TFEDREV
count,361.000000,3.610000e+02,3.610000e+02,361.000000,361.000000,3.610000e+02,3.610000e+02
mean,11.419910,8.977678e+08,1.432108e+08,10755.099972,14493.880388,1.842438e+05,1.364853e+04
std,12.392154,2.844458e+09,4.068459e+08,26375.527916,7862.091499,5.454082e+05,5.650702e+04
min,0.000000,1.431487e+06,3.724809e+05,8.580000,9560.140000,2.460000e+02,0.000000e+00
25%,4.479730,1.136355e+08,2.160969e+07,1613.240000,11902.960000,2.720200e+04,1.463000e+03
50%,8.340659,3.526923e+08,5.821769e+07,4861.630000,12964.900000,7.694900e+04,4.431000e+03
75%,14.455556,1.000056e+09,1.623661e+08,12557.030000,14218.360000,2.127670e+05,1.341800e+04
max,93.016667,5.093915e+10,7.213548e+09,453121.470000,126344.420000,9.840318e+06,1.036366e+06


We need to remove some outliers (for example, >90%) and (0%).

More plots on the dropout rate?